In [104]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

## Upsampling 

In [105]:
from sklearn.utils import resample

category_0 = data[data['TARGET_B'] == 0]
category_1 = data[data['TARGET_B'] == 1]

In [106]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))

In [107]:
print(category_0.shape)
print(category_1_oversampled.shape)

(90569, 339)
(90569, 339)


In [108]:
balanced_data = pd.concat([category_0, category_1_oversampled], axis = 0)
balanced_data = balanced_data.reset_index(drop=True)

balanced_data.shape

(181138, 339)

## X-y Split, OneHot encoding and train test

In [109]:
y = balanced_data['TARGET_B']
X = balanced_data.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(np.object)

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [110]:
print(X_train.shape)

(144910, 355)


In [111]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

## Random Forest Algorithm

In [112]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.6189565937478435
0.6129513083802584


In [113]:
predictions = clf.predict(X_test)


In [114]:
predictions

array([1, 1, 0, ..., 0, 1, 1])

In [115]:
print(len(predictions))
print(len(y_test))

36228
36228


In [116]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.61      0.61      0.61     17974
           1       0.62      0.62      0.62     18254

    accuracy                           0.61     36228
   macro avg       0.61      0.61      0.61     36228
weighted avg       0.61      0.61      0.61     36228



In [117]:
from sklearn.metrics import confusion_matrix
y_pred = list(predictions)
y_act = y_test
confusion_matrix(y_act, y_pred)

array([[10919,  7055],
       [ 6967, 11287]])

## Discussing the results

We can see that score of the model is okay high for both the train and test models. If we check other scores such as precision, recall and f1, the scores are very similar. Afterwards, we can chek the confusion matrix and we can see that the random forrest model has predicted most of the true negatives and true positives. 


However, let's try and see if other models are better, let's try with logistic regression.


## Scaling data

In [118]:
from sklearn.preprocessing import StandardScaler

transformer = StandardScaler().fit(X_train)
x_standardized_train = transformer.transform(X_train)
print(x_standardized_train.shape)
pd.DataFrame(x_standardized_train, columns=X_train.columns)

(144910, 354)


,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-0.059138,1.352235,-0.709639,0.715016,-0.369744,-0.209587,0.637644,-0.366881,0.728138,1.662304,-0.110421,0.229625,-2.032873,1.161945,1.307618,1.211256,-1.255050,-0.433602,1.848280,0.320480,-0.171768,-0.127342,-0.434081,5.098422,-0.420545,-0.477561,-0.329327,-0.182902,-0.252195,-0.257536,-0.243409,-0.210987,-0.059288,-0.370827,-0.242098,-0.101825,-0.464625,0.147964,0.588624,0.581695,0.259202,0.599355,0.584574,0.352444,-0.080310,0.099239,0.006367,-0.506122,-0.389175,-0.271317,-0.257098,0.743443,0.376835,0.637203,-0.037965,0.227030,0.252307,0.056774,-0.221200,0.539239,0.477472,0.567449,0.100189,0.494124,-0.307172,-0.220698,-0.053798,0.044623,0.122641,-0.363310,0.532210,-0.196959,-0.338051,-0.144141,0.467890,0.560522,-0.152586,-0.604198,-0.598035,-0.543677,0.043273,0.221670,-0.155982,-0.594251,-0.590598,-0.985404,-0.862950,0.092221,-0.056771,-3.433080,4.491600,3.677651,-0.105831,0.032178,-0.065587,-0.312810,0.076117,-0.023340,0.675596,0.353767,0.451661,0.199763,-0.206638,-0.211138,-0.120385,-0.504674,0.427403,-0.421813,-0.440258,-0.270162,-0.498328,-0.596511,-0.754003,-0.872722,-0.732356,-0.374642,0.160906,-0.566955,-0.388503,-0.478062,-0.439169,-2.033863,-0.476293,-0.486421,0.166997,0.768020,-0.013163,-0.448539,0.125919,-0.934985,-1.195654,-1.440348,-1.432365,-1.252101,1.423096,0.054071,-0.912618,-0.944323,-0.884463,-0.873271,-0.684417,1.255635,0.783037,0.059664,-0.725995,-0.951016,-0.808493,-0.383864,-0.001822,-0.405017,1.304206,0.900536,0.225920,-0.662646,-0.939792,-0.711552,-0.431446,-0.082241,-0.413394,0.648775,1.184026,-0.633754,1.005249,-0.291058,0.341865,-0.046281,-0.261863,-0.278653,-0.431617,-0.403157,-0.210846,-0.134804,-0.353221,1.087374,0.457003,0.119492,-0.323596,-1.127134,-0.875698,-0.434699,-1.105887,-0.485151,-0.687184,-0.364467,-0.939222,-0.438814,0.232561,0.112479,-0.005748,0.659625,0.402897,-0.285480,-0.112700,-0.591422,-0.102081,-0.430707,-0.405759,-0.328068,1.272430,0.401924,-0.051929,-0.021542,0.010055,0.117518,0.144747,-0.279823,0.378176,-0.477920,-0.394305,-0.301154,-0.763159,0.979796,-0.63

In [119]:
x_standardized_test = transformer.transform(X_test)
print(x_standardized_test.shape)
pd.DataFrame(x_standardized_test, columns=X_test.columns)

(36228, 354)


,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-0.059138,-0.016617,-1.897536,-2.272930,2.086740,-0.209587,0.550949,-0.037670,0.616933,0.972395,-0.502255,0.720722,-2.032873,-0.207052,-0.195319,-0.261890,0.839947,-0.433602,-0.644008,0.320480,-0.171768,-0.077647,-0.370298,-0.233832,1.560954,0.113058,-0.329327,1.970763,0.568509,1.111486,1.552902,0.805169,-0.059288,0.167282,-0.242098,-0.101825,-0.161642,0.147964,0.225450,0.211580,-0.015385,-0.123736,-0.001526,-0.179951,-0.935144,0.572939,1.054090,0.164035,-0.517005,-0.271317,1.599489,1.470290,-0.605700,-0.703345,-0.826088,-1.154652,0.440351,1.168760,0.326078,-0.517105,-0.862472,-0.652099,-1.006931,-0.351992,1.067697,0.864568,1.207548,0.568698,0.280056,-0.363310,-0.499075,0.148311,1.008067,1.006051,1.069078,0.787669,-0.533521,-0.730958,-0.642439,-0.641855,-0.300496,-0.274097,-0.155982,1.767011,1.687625,2.398471,2.192226,0.883595,-0.878250,0.701114,-0.819534,-0.049647,0.354694,1.032769,0.934498,0.523781,0.723695,-0.776601,-0.330295,-0.555966,-0.237402,-0.930141,-0.636759,-0.702893,-0.267900,0.347558,-0.377403,-0.421813,-0.334935,-0.270162,2.479674,2.165402,1.636518,1.245718,0.814830,1.743951,-0.646723,1.570958,1.010180,1.162940,0.735209,0.547515,-0.635789,-0.544913,-0.589517,-0.141525,-0.563158,-0.617993,-0.529043,1.822999,1.372595,0.950288,0.614298,0.815313,-1.243763,1.133299,1.335491,1.228942,1.501385,1.355190,0.433991,-1.121573,-0.718041,-1.690605,0.358831,0.697528,2.046304,3.444692,1.009714,-0.006820,-1.029605,-0.620618,-1.598187,-0.072613,0.754587,1.798445,2.836550,0.765651,-0.076296,-0.565121,0.179241,0.612427,-0.560334,-0.728997,0.778345,-0.534127,0.737498,-0.575639,-0.431617,-0.403157,-0.210846,-0.134804,-0.353221,-0.372668,-0.354205,-0.361288,1.672201,0.678079,0.637294,-0.250270,0.570583,0.104318,0.343503,-0.145905,0.603447,-0.000851,-0.319466,-0.415039,-0.005748,0.815495,-0.383117,0.196181,0.447921,0.546772,-0.102081,0.970924,-0.405759,1.695787,-0.397881,-0.356801,-0.566478,-0.541178,-0.297925,-0.226117,-0.556464,-0.279823,-0.348811,0.113053,1.160668,0.565738,1.824144,-1.116809,0.215975,1.414424,-0.70827

## Aplying a logistic regression model

In [120]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()
logistic_regression.fit(x_standardized_train, y_train)

LogisticRegression()

In [122]:
print("Test data accuracy was",logistic_regression.score(x_standardized_test,y_test))
print("Train data accuracy was",logistic_regression.score(x_standardized_train,y_train))

Test data accuracy was 0.6029038312907143
Train data accuracy was 0.6045890552756884


In [123]:
predictions = logistic_regression.predict(X_test)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [124]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.51      0.04      0.07     17974
           1       0.50      0.96      0.66     18254

    accuracy                           0.50     36228
   macro avg       0.51      0.50      0.37     36228
weighted avg       0.51      0.50      0.37     36228



In [125]:
from sklearn.metrics import confusion_matrix
y_pred = list(predictions)
y_act = y_test
confusion_matrix(y_act, y_pred)

array([[  692, 17282],
       [  652, 17602]])

## Discusing results

As we can see, our new model has a lower score and hasn't been able to predict as many true negatives and true positives. We can conclude that that the Random forest regression is a better model for this case. 

# Predicting Target D: amount donated

In [141]:
data['TARGET_D'].value_counts()

0.00     90569
10.00      941
15.00      591
20.00      577
5.00       503
         ...  
18.25        1
10.70        1
2.50         1
16.87        1
44.21        1
Name: TARGET_D, Length: 71, dtype: int64

## Filtering the data so we can predict later the amount donated by actual donors

In [165]:
data[data['TARGET_D'] > 0.0]

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
20,2,62.000000,3,8,10,2,25,40,27,11,4,1,9,2707,672,929,99,0,0,45,55,97,1,0,2,1,0,0,0,0,1,0,0,1,0,0,0,35,42,43,37,47,50,27,34,43,23,9,20,27,16,8,6,15,15,13,37,21,14,16,8,15,23,30,47,29,11,2,55,10,13,22,190,267,89,49,1,10,10,8,8,8,0,902,960,8,8,88,12,97,3,11,42,72,60,34,79,21,8,1,7,12,21,5,21,58,17,0,1,0,0,4,35,74,99,0,1,64,63,62,14,4,2,8,0,5,1,7,0,95,96,96,96,5120.0,107.0,613.0,468,501,540,580,20328,6,8,19,23,23,11,7,1,2,4,6,13,27,25,14,8,1,1,14,2,66,1,61,39,20,74,17,2,2,0,0,0,2,4,1,75,20,21,1,67,71,82,62,80,60,80,65,99,0,1,21,20,8,11,17,1,1,10,0,5,3,2,1,0,0,3,23,5,2,4,12,7,3,2,1,14,12,7,5,11,4,1,4,70,10,0,140,5,3,24,21,9,32,7,3,23,3,16,7,1,2,0,12,25,3,40,20,27,7,0,0,2,12,0,0,3,1,4,1,0,0,0,4,0,1,58,95,1,0,4,95,69,23,4,13,2,17,35,91,99,1,12,3,93,0,5,1,0,1,99,1,97,99,99,9,2,6,3,26,65,5,12,61.00,15,10,2.00,7.0,5.0,12,4.066667,82943,1,3,3,other,12,H,F,3,L,D,A,S,1,87,1,36,1,88,1,94,4,96,3,87,1,1,4.0
30,0,61.611649,5,9,0,1,37,58,16,8,1,5,9,2147,591,640,99,0,0,49,51,94,2,0,3,5,1,0,1,1,0,0,0,3,0,0,2,29,36,37,27,38,40,37,46,37,17,8,34,34,15,6,2,1,18,21,31,19,11,5,0,3,7,23,71,47,15,6,77,5,1,17,286,334,99,99,0,0,0,0,0,0,0,1002,1166,7,5,97,3,96,4,3,63,92,88,59,97,3,4,1,3,5,4,1,30,60,4,1,2,0,4,15,50,94,99,2,0,91,70,72,20,4,0,0,0,3,0,0,0,78,83,99,99,2920.0,201.0,618.0,633,638,652,663,19703,2,5,1,17,48,19,7,0,1,3,4,0,16,51,19,7,0,1,5,0,65,3,74,26,25,89,9,0,0,0,0,0,1,2,1,82,34,42,22,70,82,98,67,97,66,68,61,99,99,0,34,23,4,10,12,0,2,9,0,4,1,1,0,0,4,2,25,5,2,4,8,8,8,0,3,10,8,9,4,8,1,5,5,76,4,1,160,0,1,12,25,10,39,13,5,33,6,25,7,1,1,0,19,37,2,58,11,16,10,0,4,1,15,0,0,3,2,0,2,0,0,0,1,0,3,48,97,1,1,1,99,91,24,3,6,6,57,85,95,95,5,0,0,75,0,20,0,0,5,96,4,90,99,99,12,4,1,6,10,24,0,2,68.00,11,6,2.00,10.0,7.0,9,6.181818,190313,1,3,14,TX,35,H,M,3,L,D,A,T,1,90,1,0,2,90,4,93,1,95,12,90,4,1,7.0
45,0,66.000000,5,9,5,0,33,24,39,6,5,1,9,2160,683,900,89,0,11,48,52,99,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,42,47,49,40,48,50,20,33,42,25,7,14,23,21,17,13,5,14,13,33,22,18,24,7,22,22,43,35,19,6,1,72,7,5,16,165,240,69,58,1,28,27,4,0,0,0,1282,1399,5,4,96

In [166]:
filtered_data = data[data['TARGET_D'] > 0.0]
filtered_data = filtered_data.reset_index(drop=True)

In [167]:
filtered_data

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,2,62.000000,3,8,10,2,25,40,27,11,4,1,9,2707,672,929,99,0,0,45,55,97,1,0,2,1,0,0,0,0,1,0,0,1,0,0,0,35,42,43,37,47,50,27,34,43,23,9,20,27,16,8,6,15,15,13,37,21,14,16,8,15,23,30,47,29,11,2,55,10,13,22,190,267,89,49,1,10,10,8,8,8,0,902,960,8,8,88,12,97,3,11,42,72,60,34,79,21,8,1,7,12,21,5,21,58,17,0,1,0,0,4,35,74,99,0,1,64,63,62,14,4,2,8,0,5,1,7,0,95,96,96,96,5120.0,107.0,613.0,468,501,540,580,20328,6,8,19,23,23,11,7,1,2,4,6,13,27,25,14,8,1,1,14,2,66,1,61,39,20,74,17,2,2,0,0,0,2,4,1,75,20,21,1,67,71,82,62,80,60,80,65,99,0,1,21,20,8,11,17,1,1,10,0,5,3,2,1,0,0,3,23,5,2,4,12,7,3,2,1,14,12,7,5,11,4,1,4,70,10,0,140,5,3,24,21,9,32,7,3,23,3,16,7,1,2,0,12,25,3,40,20,27,7,0,0,2,12,0,0,3,1,4,1,0,0,0,4,0,1,58,95,1,0,4,95,69,23,4,13,2,17,35,91,99,1,12,3,93,0,5,1,0,1,99,1,97,99,99,9,2,6,3,26,65,5,12,61.00,15,10,2.00,7.0,5.0,12,4.066667,82943,1,3,3,other,12,H,F,3,L,D,A,S,1,87,1,36,1,88,1,94,4,96,3,87,1,1,4.0
1,0,61.611649,5,9,0,1,37,58,16,8,1,5,9,2147,591,640,99,0,0,49,51,94,2,0,3,5,1,0,1,1,0,0,0,3,0,0,2,29,36,37,27,38,40,37,46,37,17,8,34,34,15,6,2,1,18,21,31,19,11,5,0,3,7,23,71,47,15,6,77,5,1,17,286,334,99,99,0,0,0,0,0,0,0,1002,1166,7,5,97,3,96,4,3,63,92,88,59,97,3,4,1,3,5,4,1,30,60,4,1,2,0,4,15,50,94,99,2,0,91,70,72,20,4,0,0,0,3,0,0,0,78,83,99,99,2920.0,201.0,618.0,633,638,652,663,19703,2,5,1,17,48,19,7,0,1,3,4,0,16,51,19,7,0,1,5,0,65,3,74,26,25,89,9,0,0,0,0,0,1,2,1,82,34,42,22,70,82,98,67,97,66,68,61,99,99,0,34,23,4,10,12,0,2,9,0,4,1,1,0,0,4,2,25,5,2,4,8,8,8,0,3,10,8,9,4,8,1,5,5,76,4,1,160,0,1,12,25,10,39,13,5,33,6,25,7,1,1,0,19,37,2,58,11,16,10,0,4,1,15,0,0,3,2,0,2,0,0,0,1,0,3,48,97,1,1,1,99,91,24,3,6,6,57,85,95,95,5,0,0,75,0,20,0,0,5,96,4,90,99,99,12,4,1,6,10,24,0,2,68.00,11,6,2.00,10.0,7.0,9,6.181818,190313,1,3,14,TX,35,H,M,3,L,D,A,T,1,90,1,0,2,90,4,93,1,95,12,90,4,1,7.0
2,0,66.000000,5,9,5,0,33,24,39,6,5,1,9,2160,683,900,89,0,11,48,52,99,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,42,47,49,40,48,50,20,33,42,25,7,14,23,21,17,13,5,14,13,33,22,18,24,7,22,22,43,35,19,6,1,72,7,5,16,165,240,69,58,1,28,27,4,0,0,0,1282,1399,5,4,96,4,

## X-y split, One-hot encoding and train test split

In [168]:
y = filtered_data['TARGET_D']
X = filtered_data.drop(['TARGET_D'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(np.object)

encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Scaling the data

In [169]:
transformer = StandardScaler().fit(X_train)
x_standardized_train = transformer.transform(X_train)
print(x_standardized_train.shape)
pd.DataFrame(x_standardized_train, columns=X_train.columns)

(3874, 355)


,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-0.064328,-0.039494,0.435324,0.710441,-0.373194,-0.003400,0.883335,-0.546749,0.752540,0.300350,-0.306807,-0.760647,0.767177,-0.200109,-0.200152,-0.253812,-1.272599,-0.425380,1.850380,0.320261,-0.182134,0.609169,-0.348496,-0.259786,-0.437465,-0.412862,-0.332049,-0.193226,-0.281040,-0.276578,-0.266100,-0.222798,-0.063847,-0.470755,0.634022,-0.103927,-0.173173,-0.599195,-0.767014,-0.922452,-0.869182,-0.585590,-0.763586,1.044584,0.894479,-0.061584,-1.090494,-0.478462,1.425421,0.991992,-0.735120,-0.976081,-0.626252,-0.408844,0.536795,1.079540,0.062133,-0.394666,-0.803100,-0.455872,-0.330922,-0.514569,-0.834951,-0.641128,1.089689,1.163744,0.791008,0.079180,0.975427,-1.276945,-0.289236,-0.524905,1.142240,0.940778,1.061894,1.121052,-0.133374,-0.684910,-0.686608,-0.641451,-0.301925,-0.269360,-0.163356,0.180931,0.166050,1.067507,0.825414,0.920335,-0.917737,-0.080717,0.169903,1.018947,1.226397,0.771897,0.990544,1.531105,0.733007,-0.781457,-0.508485,0.387339,-0.688124,-0.645796,-0.856890,-0.476108,1.659160,0.548535,-0.564097,-0.407749,-0.424575,-0.251826,-0.321974,0.132486,1.293125,1.140652,0.782509,-0.342790,-0.654623,0.665734,0.339577,0.571843,0.773548,-0.291399,-0.549961,-0.545289,-0.589923,-0.397235,-0.416137,-0.615049,-0.539032,1.453747,1.153568,0.830645,0.723835,0.685643,-1.245445,-1.478938,0.492611,0.369126,0.227931,0.144729,-0.202899,-0.600897,-0.825656,0.231734,0.618277,1.488955,-0.340083,0.180662,-0.532552,-0.406446,-0.753840,-0.496547,0.368595,0.152962,1.479986,-0.305293,0.040881,-0.534569,-0.419815,0.204735,-0.651442,-0.149591,-0.872198,0.055313,-0.012104,-0.843633,0.733265,-0.398805,-0.232270,-0.152599,-0.220338,-0.122652,-0.369330,-0.723892,-0.109499,0.820913,1.105606,0.700290,1.986085,3.935896,-0.774013,0.394833,0.543882,0.216881,0.725043,0.136187,-0.218267,0.145759,0.465875,0.818892,-0.490102,-0.439016,-0.428374,-0.211684,-0.844785,-0.245044,0.517873,0.62323,0.135327,-0.170914,1.341710,0.361786,1.264009,-0.206202,-0.040551,-0.260791,1.081402,-0.366313,0.876768,0.605937,-0.113238,0.174771,0.405629,-

In [170]:
x_standardized_test = transformer.transform(X_test)
print(x_standardized_test.shape)
pd.DataFrame(x_standardized_test, columns=X_test.columns)

(969, 355)


,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-0.064328,0.280577,-0.164194,0.710441,-0.373194,-0.003400,-0.673947,0.440928,-0.009939,-1.088972,0.855041,0.503726,0.767177,0.384478,0.316395,0.734010,0.830473,-0.425380,-0.642468,-0.059329,0.185264,-2.572103,-0.348496,-0.259786,8.438616,-0.336100,-0.332049,10.368413,5.429177,2.245700,7.355879,3.166409,4.677539,-0.376798,0.634022,-0.103927,0.124200,0.230469,-0.282972,-0.426879,0.508057,-0.146567,-0.167729,-1.614844,0.522382,-0.551136,0.200466,-0.312268,0.525976,0.353968,-0.033851,-0.247265,0.023176,-0.259225,0.536795,0.219666,-0.322379,-0.849570,0.545245,-0.305998,-0.197701,-0.287206,1.368109,0.696303,-1.459873,-1.475003,-1.164138,-0.761980,-1.266996,1.481285,-0.289236,1.324870,-1.169286,-1.309430,-2.585818,-2.349780,-0.133374,3.168061,3.354269,3.617594,-0.301925,-0.269360,-0.163356,1.873860,1.995887,0.651793,0.825414,-1.290446,1.368325,0.236501,-0.224349,-0.247794,-1.461898,-1.582534,-1.546726,-1.476501,-1.302025,1.549141,-0.293481,-0.549335,-0.441062,2.021982,1.296732,1.988734,-2.229096,-1.832444,-0.724440,-0.407749,-0.310811,-0.251826,2.129318,2.012413,1.531308,1.140652,0.782509,2.153765,4.049380,-1.980353,-2.111061,-2.269943,-1.255380,1.500475,0.428439,4.029494,-0.589923,-1.060148,0.145038,2.040935,-0.539032,1.181623,1.153568,0.954164,0.766375,-0.116859,-1.310638,0.586175,-0.096126,0.151223,0.100838,0.338219,0.493807,-0.458402,0.037661,0.869107,0.209329,0.052468,-0.010383,0.180662,0.463396,-0.217195,-0.409334,-0.388715,0.498384,-0.555295,0.540228,0.423602,0.290284,0.733341,-0.258598,-0.501978,-0.302746,0.578673,-0.321800,0.800000,-0.755395,1.000305,-1.387281,0.639283,2.188210,2.609872,-0.220338,1.215807,2.701020,0.873527,-0.633200,-0.369924,-0.896392,-0.389738,-0.588254,-0.624575,-0.011104,0.617394,0.127986,1.095138,0.387721,1.235942,0.961389,1.037252,0.791475,0.818892,-0.609800,1.005868,0.413025,0.124684,-0.119815,0.281467,-0.381786,-0.31041,0.473121,-0.351328,-0.891827,-0.699332,-0.284230,-0.552314,-0.377667,-0.260791,-0.829798,-1.061852,1.194773,-0.301271,-0.435326,-0.149182,0.405629,0.426244,1.

## Regression Models

In [171]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

In [172]:
model1 = DecisionTreeRegressor()
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
from sklearn.neighbors import KNeighborsRegressor
model3 = KNeighborsRegressor()

In [173]:
import numpy as np

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=10))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': -0.0787273161867609, 'Linear Regression': 0.13588532801405923, 'KNN': -0.19218573367760586}


In [174]:
predictions = {}
i=0
for model in model_pipeline:
    predicted_results = cross_val_predict(model, X_train, y_train, cv=10)
    predictions[model_names[i]] = predicted_results
    i = i+1
print(predictions)

{'Regression Tree': array([27., 10.,  5., ..., 11.,  7., 20.]), 'Linear Regression': array([33.75775119, 17.41329825,  7.51983949, ..., 13.93004754,
       10.71319175, 26.64678037]), 'KNN': array([15.6, 15.4, 15.8, ..., 13.2, 12. , 20. ])}


In [184]:
list(predictions['Linear Regression'])

[33.757751186002146,
 17.413298254792902,
 7.519839493270583,
 19.993012426023906,
 31.743140188505258,
 6.938956821315159,
 10.635905805457124,
 10.274092023008306,
 8.203007296303959,
 16.98992100961857,
 10.763072858646684,
 20.987016104437284,
 29.367072517626898,
 18.288260285589978,
 4.660027899827295,
 14.984978154754842,
 16.266036959134574,
 16.202444171836632,
 15.300853393420134,
 28.149973185436835,
 8.764886707176426,
 31.776614057300467,
 15.94257989582685,
 17.96928468643864,
 18.969662943439378,
 34.024639239583514,
 7.982893394611349,
 13.05125728072643,
 39.47411593358059,
 4.651459695833143,
 4.476552477952978,
 8.823087466398384,
 19.193062538938225,
 11.708257115474769,
 13.753482897297634,
 21.643508047843966,
 10.829841352040093,
 10.617797713143812,
 10.05242735573673,
 3.564192435249563,
 30.357996000943956,
 18.117689208049317,
 185.72755969405677,
 20.384205048868246,
 11.614793546274418,
 12.98420680855209,
 22.414443235916035,
 13.847414375151125,
 12.25599

In [185]:
len(list(predictions['Linear Regression']))

3632

In [186]:
len(data)

95412

In [192]:
full_cost = 95412*0.6
print(full_cost)

57247.2


In [194]:
donors_cost = 3632*0.6
print(donors_cost)

2179.2


In [196]:
sum = 0 

for i in range(0, len(predictions['Linear Regression'])):  
   sum = sum + predictions['Linear Regression'][i]  

In [197]:
print("Sum of all the donations: " + str(sum));

Sum of all the donations: 56896.76800108094


As we can observe, the cost of sending a gift to every member of the data set is higher thant he sum of the predicted donotions. Therefore, in order to not loose moeny, the organization should focus on sending gifts to only past donors. 